In [ ]:
def ga_scraper(login, password, accountName, downloadPath,
              startDate, endDate, lang):
    return visitorsFilesList

In [1]:
# importing libraries
import selenium
import time
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd 

In [10]:
# setting login, password etc.
login =''
password = ''
audience = 'АУДИТОРИЯ'
userStastics = 'Статистика по пользователям'
startDate = '01.01.2017'
endDate = '30.01.2017'
#downloadPath = 'C:/Users/kudinov/Documents/Google_Analytics/Sandbox/jsons'
downloadPath = 'C:/Users/Alexander/Downloads/json'

In [11]:
# constructng a list of json file names with visitors data
clientDataFileList = ['user-report-export.json']
for i in range(1, 101):
    clientDataFileList.append('user-report-export ('+str(i)+').json')


In [12]:
# changing default download directory

chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : downloadPath}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path=r"chromedriver.exe", chrome_options=chromeOptions)

# going to the home web page
#driver = webdriver.Chrome(executable_path=r"chromedriver.exe")
driver.get("https://analytics.google.com/analytics/web/")
logFile = open('log_file.txt', 'w')
logFile.writelines('Going to Google Analytics web page\n')
logFile.writelines('Parameters:\n')
logFile.writelines('{0}{1}'.format('Start date = ', startDate))
logFile.writelines('{0}{1}'.format('End date = ', endDate))



# checking that the opened page is desired
#assert "Analytics" in driver.title



# log-in (passing login) and clicking next (to the password) 
driver.find_element_by_id("identifierId").clear()
driver.find_element_by_id("identifierId").send_keys(login)
driver.find_element_by_id("identifierNext").click()
logFile.writelines('Logging in\n')


# passing password 
## Unworkable code -- driver.find_element_by_id("passwrd").send_keys(password)
## Unworkable code -- driver.find_element_by_id("passwordNext").click()driver.find_element_by_name("password").send_keys(password)
try:
    element = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.NAME, 'password'))
    )
except TimeoutException:
    logFile.writelines('The page for entering password has not opened\n')
    driver.quit()
    logFile.close()

logFile.writelines('Sending password\n')
driver.find_element_by_name("password").send_keys(password)
element = driver.find_element_by_id('passwordNext')
driver.execute_script("arguments[0].click();", element)

# navigating to the target web page, step 1
try:
    element = WebDriverWait(driver, 60).until(
        EC.text_to_be_present_in_element((By.PARTIAL_LINK_TEXT, audience), audience)
    )
except TimeoutException:
    logFile.writelines('The page with right menu Audience (Аудитория) has not opened\n')
    driver.quit()
    logFile.close()

logFile.writelines('Opening menu Audience (Аудитория)\n')
driver.find_element_by_partial_link_text(audience).click()

# navigating to target web page, step 2
try:
    element = WebDriverWait(driver, 60).until(
        EC.text_to_be_present_in_element((By.PARTIAL_LINK_TEXT, userStastics), userStastics)
    )
except TimeoutException:
    logFile.writelines('The page with right submenu User Statistics (Статистика по пользователям) has not opened\n')
    driver.quit()
    logFile.close()

logFile.writelines('Opening submenu User Statistics (Статистика по пользователям)\n')
driver.find_element_by_partial_link_text(userStastics).click()

# switching to the iframe by its name (galaxy) - that contains the right part of the page
try:
    element = WebDriverWait(driver, 60).until(
        EC.visibility_of_element_located((By.NAME, 'galaxy'))
    )
except TimeoutException:
    logFile.writelines('The page with User Statistics (Статистика по пользователям) has not loaded\n')
    driver.quit()
    logFile.close()

driver.switch_to.frame(driver.find_element_by_name('galaxy'))

# activating datepicker by finding it by classs name and clicking with executing script
try:
    element = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.CLASS_NAME, '_GArn'))
    )
except TimeoutException:
    logFile.writelines('Failed to activate calendar\n')
    driver.quit()
    logFile.close()

logFile.writelines('Activating calendar\n')
driver.find_element_by_class_name('_GArn').click()

# selecting start and end dates
driver.find_element_by_class_name('ID-datecontrol-primary-start').clear()
driver.find_element_by_class_name('ID-datecontrol-primary-start').send_keys(startDate)
driver.find_element_by_class_name('ID-datecontrol-primary-end').clear()
driver.find_element_by_class_name('ID-datecontrol-primary-end').send_keys(endDate)
driver.find_element_by_class_name('ID-apply').click()

# displaying all the rows on the page
try:
    element = WebDriverWait(driver, 30).until(
        EC.visibility_of_element_located((By.CLASS_NAME, 'ACTION-toggleRowShow'))
    )
except TimeoutException:
    logFile.writelines('Failed to set 5000 rows at the page\n')
    driver.quit()
    logFile.close()

logFile.writelines('Setting 5000 rows in the table on the page to be visible\n')
driver.find_element_by_class_name("ACTION-toggleRowShow").send_keys(5000)

# downloading the top level summary table "Статистика по пользователям"
time.sleep(5)
driver.find_element_by_class_name("ID-exportControlButton").click()
li = driver.find_elements_by_class_name("ACTION-export")
li[2].click()

# getting number of visitors' IDs from the summary table
elem = driver.find_elements_by_class_name("C_USER_LIST_TEXT_DIV")
static_len = len(elem)

# saving url of the page with the top level summary table "Статистика по пользователям"
summaryTableURL = driver.current_url

# renaming the downloaded file with the summary table to ClientSummary.xlsx
os.rename('{0}{2}{1}'.format(downloadPath, os.listdir(downloadPath)[0],'/'),
          '{0}{2}{1}'.format(downloadPath, 'ClientSummary.xlsx','/'))


# loading the 1st column (IDs) from the summary table (ClientSummary.xlsx) dataFrame clientsID
clientsID = pd.read_excel(io = '{0}{2}{1}'.format(downloadPath, 'ClientSummary.xlsx','/'), sheet_name = 'Набор данных1',
                             index_col = [0])

# creating the log file and storing # of visitors
logFile.writelines('\n)
logFile.writelines('# of visitors:\n')
logFile.writelines('{0}'.format(len(clientsID)))
logFile.writelines(' \n')
logFile.writelines(' \n')


for number in range(0,static_len):
    logFile.writelines('# of step, ID:\n')
    if number in [101*i for i in range(1,50)]:
        for k in range(0,101):
            os.rename('{0}{2}{1}'.format(downloadPath, clientDataFileList[k],'/'),
                      '{0}{2}{1}{3}'.format(downloadPath, clientsID.index[k+number-101],'/','.json'))        
    time.sleep(2)
    try:
        element = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.CLASS_NAME, 'C_USER_LIST_TEXT_DIV'))    )
    except TimeoutException:
        logFile.writelines('5000 rows in the table at the page has not been loaded\n')
        driver.quit()
        logFile.close()

    elem = driver.find_elements_by_class_name("C_USER_LIST_TEXT_DIV")
    elem[number].click()
#    time.sleep(5)
    try:
        item = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.CLASS_NAME, '_GAGW'))    )
    except TimeoutException:
        logFile.writelines('{0}{1}{2}'.format('The page of the visitor with ID = ',
                                           clientsID[number],
                                           ' has not opened to download the visitor data\n')
        driver.quit()
        logFile.close()

    item = driver.find_elements_by_class_name("_GAGW")
    item[5].click()
    logFile.writelines('{0}{2}{1}{3}'.format(number,clientsID[number],'. ', '.\n'))
    driver.get(summaryTableURL)



#    driver.back()
#    driver.back()
#    driver.back()
#    driver.back()
#    driver.back()
    try:
        element = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.NAME, 'galaxy'))    )
    except TimeoutException:
        logFile.writelines('5000 rows in the table at the page has not been loaded\n')
        driver.quit()
        logFile.close()
        
#    time.sleep(15)
    driver.switch_to.frame(driver.find_element_by_name('galaxy'))

# closing the log file
logFile.close()

driver.close()

Sobyakin asshole. Once


TimeoutException: Message: 


In [ ]:
driver.close()

In [8]:
help(selenium.webdriver.common.by)

Help on module selenium.webdriver.common.by in selenium.webdriver.common:

NAME
    selenium.webdriver.common.by - The By implementation.

CLASSES
    builtins.object
        By
    
    class By(builtins.object)
     |  Set of supported locator strategies.
     |  
     |  Data descriptors defined here:
     |  
     |  __dict__
     |      dictionary for instance variables (if defined)
     |  
     |  __weakref__
     |      list of weak references to the object (if defined)
     |  
     |  ----------------------------------------------------------------------
     |  Data and other attributes defined here:
     |  
     |  CLASS_NAME = 'class name'
     |  
     |  CSS_SELECTOR = 'css selector'
     |  
     |  ID = 'id'
     |  
     |  LINK_TEXT = 'link text'
     |  
     |  NAME = 'name'
     |  
     |  PARTIAL_LINK_TEXT = 'partial link text'
     |  
     |  TAG_NAME = 'tag name'
     |  
     |  XPATH = 'xpath'

FILE
    c:\programdata\anaconda3\lib\site-packages\selenium\w

In [9]:
driver.find_element_by_class_name


NameError: name 'driver' is not defined

In [2]:
help(WebDriverWait)

Help on class WebDriverWait in module selenium.webdriver.support.wait:

class WebDriverWait(builtins.object)
 |  Methods defined here:
 |  
 |  __init__(self, driver, timeout, poll_frequency=0.5, ignored_exceptions=None)
 |      Constructor, takes a WebDriver instance and timeout in seconds.
 |      
 |      :Args:
 |       - driver - Instance of WebDriver (Ie, Firefox, Chrome or Remote)
 |       - timeout - Number of seconds before timing out
 |       - poll_frequency - sleep interval between calls
 |         By default, it is 0.5 second.
 |       - ignored_exceptions - iterable structure of exception classes ignored during calls.
 |         By default, it contains NoSuchElementException only.
 |      
 |      Example:
 |       from selenium.webdriver.support.ui import WebDriverWait 
 |      
 |       element = WebDriverWait(driver, 10).until(lambda x: x.find_element_by_id("someId")) 
 |      
 |       is_disappeared = WebDriverWait(driver, 30, 1, (ElementNotVisibleException)).\ 
 |  

In [44]:

clientsID.head(3)

,Сеансы,Сред. длительность сеанса,Показатель отказов,Доход,Транзакции,Коэффициент конверсии цели
Идентификатор клиента,,,,,,
262307731.1464884920,25,601.84000,0.120000,0,0,0
1370805766.1445343102,21,2.52381,0.952381,0,0,0
1754705724.1482953893,14,0.00000,1.000000,0,0,0


In [13]:
"""
dowloadedFileName = '{0}{5}{1}{5}{3}{5}{4}{2}{6}'.format('Google Analytics', accountName,
                              startDate[6:]+startDate[3:5]+startDate[0:2],
                              endDate[6:]+endDate[3:5]+endDate[0:2],
                              ' - ',' ','.xlsx')
"""                              

"\ndowloadedFileName = '{0}{5}{1}{5}{3}{5}{4}{2}{6}'.format('Google Analytics', accountName,\n                              startDate[6:]+startDate[3:5]+startDate[0:2],\n                              endDate[6:]+endDate[3:5]+endDate[0:2],\n                              ' - ',' ','.xlsx')\n"

In [ ]:
logFile = open('log_file.txt', 'w')
logFile.writelines('# of visitors:/n')
logFile.writelines(len(clientsID))
logFile.writelines('/n')
logFile.close()

In [ ]:
# j is a number of iterations over elements in
if j in [101*i for i in range(1,50)]:
    for k in range(0,100):
        os.rename(clientDataFileList[k],clientsID.iat[0,k+j-101])

In [15]:
logFile.writelines('# of elements in by selenium: ')